In [1]:
import os
import pandas as pd
import glob
from datetime import datetime
from bs4 import BeautifulSoup

In [3]:
selection = 'marketinsights'
directory = "./utils/data/Scraped News/"
file_pattern = f"{selection}_data_*.csv"
files = glob.glob(os.path.join(directory, file_pattern))
print(files)
latest_file = max(files, key=os.path.getctime)
print(latest_file)

['./utils/data/Scraped News/marketinsights_data_2024-07-01.csv']
./utils/data/Scraped News/marketinsights_data_2024-07-01.csv


In [4]:
from utils.marketinsights import safe_literal_eval, process_tables
df = pd.read_csv(latest_file)
df.head()


,title,link,ticker,date,source,Article content,raw,tables,Time,Executives,Shareholders
0,Cordiant Digital Infrastructure Limited Announ...,https://www.marketscreener.com/quote/stock/COR...,CORD,02:00am,S&P Capital IQ,Cordiant Digital Infrastructure Limited announ...,"<!DOCTYPE html>\n<html lang=""en-US"" prefix=""og...",{'Managers': {'Managers': {0: 'David Charles B...,2024-07-01 02:00:00,"[['David Charles Bonar', 'Treasurer', '53', '2...","[['Newton Investment Management Ltd.', '37,855..."
1,Certain Common Stock of Public Joint-Stock Com...,https://www.marketscreener.com/quote/stock/SOV...,SVCB,2024-06-30,S&P Capital IQ,Certain Common Stock of Public Joint-Stock Com...,"<!DOCTYPE html>\n<html lang=""en-US"" prefix=""og...",{'Managers': {'Managers': {0: 'Dmitry Vladimir...,2024-06-30 00:00:00,"[['Aleksey Valeryevich Panferov', 'Corporate O...",[]
2,Certain A Shares of Chengdu Qinchuan IoT Techn...,https://www.marketscreener.com/quote/stock/CHE...,688528,2024-06-30,S&P Capital IQ,Certain A Shares of Chengdu Qinchuan IoT Techn...,"<!DOCTYPE html>\n<html lang=""en-US"" prefix=""og...","{'Managers': {'Managers': {0: 'Ze Hua Shao', 1...",2024-06-30 00:00:00,"[['Jing Zhang', 'Chief Administrative Officer'...","[['Gfund Management Co., Ltd.', '1,454,100', '..."
3,SAUDI ARAMCO - ARAMCO'S STRATEGIC GAS EXPANSIO...,https://www.marketscreener.com/quote/stock/ARA...,2222,Jun. 30,Reuters,SAUDI ARAMCO - ARAMCO'S STRATEGIC GAS EXPANSIO...,"<!DOCTYPE html>\n<html lang=""en-US"" prefix=""og...",{'Managers': {'Managers': {0: 'Amin Al-Nasser'...,2024-06-30 00:00:00,"[['Ahmad Al-Khowaiter', 'Chief Tech/Sci/R&D Of...","[['Government of Saudi Arabia', '198,890,120,0..."
4,Certain Employee Stock Options of Corebridge F...,https://www.marketscreener.com/quote/stock/COR...,CRBG,Jun. 29,S&P Capital IQ,Certain Employee Stock Options of Corebridge F...,"<!DOCTYPE html>\n<html lang=""en-US"" prefix=""og...","{'Managers': {'Managers': {0: 'Kevin Hogan', 1...",2024-06-29 00:00:00,"[['Adam Burk', 'Director/Board Member', '47', ...","[['AMERICAN INTERNATIONAL GROUP, INC.', '294,2..."


In [82]:
html_content = df['raw'][10]
def get_industry(html_content):
    soup = BeautifulSoup(html_content, 'html.parser')

    # Locate all card-header divs
    card_headers = soup.find_all('div', class_='card-header')

    # Iterate through each card-header div and extract the required information
    for card_header in card_headers:
        # Get the title text from the card-header div
        title_text = card_header.get_text(strip=True)
        if title_text == 'Sector':
            # Find the next sibling card-content div
            card_content = card_header.find_next_sibling('div', class_='card-content')
            
            if card_content:
                # Get the sector details from the card-content div
                sector_details = list(map(lambda x: x.strip(), card_content.get_text().split('\n')))
                actual = ''
                for sector in sector_details:
                    if sector:
                        actual = sector
    return actual



Real Estate
Real Estate
Real Estate Operations
Real Estate Development & Operations
Other Real Estate Development & Operations
Other Real Estate Development & Operations


In [ ]:
def get_contact_information(html_content):
    result = {}
    soup = BeautifulSoup(html_content, 'html.parser')
    # Find the div with class 'card-content'
    contact_info_div = soup.find_all('div', class_ = 'card mb-15')
    for div in contact_info_div:
    # Extract the paragraphs within this div
        contact_info_paragraphs = div.find_all('p', class_='m-0')
        if contact_info_paragraphs:
            
            company_name = contact_info_paragraphs[0].text
            address_line_1 = contact_info_paragraphs[1].text
            address_line_2 = contact_info_paragraphs[2].text
            phone_number = contact_info_paragraphs[3].text

            # Extract the website URL
            website = div.find('a', class_='m-0')['href']

        # Print the extracted information
        result['Company Name'] = company_name
        result['Address Line 1'] = address_line_1
        result['Address Line 2'] = address_line_2
        result['Phone Number'] = phone_number
        result['Website'] = website
    return result